### Carga da lista de gêneros dos filmes favoritos no TMDB

In [0]:
import json

#leitura do arquivo json com Pyspark
pyspark = spark.read.json('/temp/TMDB/ListaGenerosFilmesTMDB.json')
display(pyspark)

genres
"List(List(28, Ação), List(12, Aventura), List(16, Animação), List(35, Comédia), List(80, Crime), List(99, Documentário), List(18, Drama), List(10751, Família), List(14, Fantasia), List(36, História), List(27, Terror), List(10402, Música), List(9648, Mistério), List(10749, Romance), List(878, Ficção científica), List(10770, Cinema TV), List(53, Thriller), List(10752, Guerra), List(37, Faroeste))"


In [0]:
#consultando file system
file = dbutils.fs.ls(f"/temp/TMDB/ListaGenerosFilmesTMDB.json")
display (file)

path,name,size,modificationTime
dbfs:/temp/TMDB/ListaGenerosFilmesTMDB.json,ListaGenerosFilmesTMDB.json,698,1709231980000


In [0]:
%sql
--consultando arquivo json em formato de tabela com SparkSQL (saída parecida com PySpark)
SELECT * FROM json.`dbfs:/temp/TMDB/ListaGenerosFilmesTMDB.json`

genres
"List(List(28, Ação), List(12, Aventura), List(16, Animação), List(35, Comédia), List(80, Crime), List(99, Documentário), List(18, Drama), List(10751, Família), List(14, Fantasia), List(36, História), List(27, Terror), List(10402, Música), List(9648, Mistério), List(10749, Romance), List(878, Ficção científica), List(10770, Cinema TV), List(53, Thriller), List(10752, Guerra), List(37, Faroeste))"


In [0]:
%sql 
--criando database
CREATE SCHEMA IF NOT EXISTS DBTMDB;

In [0]:
%sql 
--criando view/tabela de referência com dados brutos a partir do json de gêneros 

CREATE OR REPLACE GLOBAL TEMP VIEW TMDB_GENEROS_VW
USING json
OPTIONS (path="dbfs:/temp/TMDB/ListaGenerosFilmesTMDB.json");



In [0]:
%sql
--para visualizar uma view gloal temporária é necessário usar o owner global_temp
SELECT * FROM GLOBAL_TEMP.TMDB_GENEROS_VW;

genres
"List(List(28, Ação), List(12, Aventura), List(16, Animação), List(35, Comédia), List(80, Crime), List(99, Documentário), List(18, Drama), List(10751, Família), List(14, Fantasia), List(36, História), List(27, Terror), List(10402, Música), List(9648, Mistério), List(10749, Romance), List(878, Ficção científica), List(10770, Cinema TV), List(53, Thriller), List(10752, Guerra), List(37, Faroeste))"


In [0]:
%sql
DESCRIBE GLOBAL_TEMP.TMDB_GENEROS_VW;

col_name,data_type,comment
genres,array>,null


In [0]:
%sql
--usando a função explode do SparkSQL para transformar cada posição do array da lista de filmes em uma linha na tabela
SELECT explode(genres)
FROM GLOBAL_TEMP.TMDB_GENEROS_VW;

col
"List(28, Ação)"
"List(12, Aventura)"
"List(16, Animação)"
"List(35, Comédia)"
"List(80, Crime)"
"List(99, Documentário)"
"List(18, Drama)"
"List(10751, Família)"
"List(14, Fantasia)"
"List(36, História)"


In [0]:
%sql  
--criando view temporária para popular as linhas com elementos do array

CREATE OR REPLACE GLOBAL TEMP VIEW TMDB_GENEROS_VW2 AS 
SELECT explode(genres)
FROM GLOBAL_TEMP.TMDB_GENEROS_VW;

SELECT * FROM GLOBAL_TEMP.TMDB_GENEROS_VW2;
DESCRIBE GLOBAL_TEMP.TMDB_GENEROS_VW2;

col_name,data_type,comment
col,struct,null


In [0]:
%sql
--ordenando gêneros por id
SELECT 
COL.ID AS ID,
COL.NAME AS NAME
FROM GLOBAL_TEMP.TMDB_GENEROS_VW2
ORDER BY ID;

ID,NAME
12,Aventura
14,Fantasia
16,Animação
18,Drama
27,Terror
28,Ação
35,Comédia
36,História
37,Faroeste
53,Thriller


In [0]:
%sql
--Criação de tabela de gêneros, que será utilizada como referência para a tabela de filmes
USE DBTMDB;

CREATE OR REPLACE TABLE TMDB_GENEROS_REF AS 
SELECT 
COL.ID AS ID,
COL.NAME AS NAME
FROM GLOBAL_TEMP.TMDB_GENEROS_VW2;



num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM TMDB_GENEROS_REF ORDER BY ID;

ID,NAME
12,Aventura
14,Fantasia
16,Animação
18,Drama
27,Terror
28,Ação
35,Comédia
36,História
37,Faroeste
53,Thriller
